# Ridge Regression with dimensionality reduction

The purpose of this notebook is to define an architecture from scratch to perform a supervised dimensionality reduction with a linear regression layer.

The previous architecture, defined in Plave Armonized Variational Inference (PAVI), uses Principal Component Analysis (PCA) to reduce the dimensionality. The main aim of PCA is to find orthogonal projections of data that maximize the variance of the data projected onto them. In theory, PCA produces the same number of principal components as there are features in the training dataset.

The PCA is based on the Singular Values Decomposition (SVD)

The SVD decomposes the matrix X into three matrices:

U: Eigenvector matrix of the covariance matrix of X.

Σ: Diagonal matrix of singular values, representing the magnitudes of the eigenvectors.

V: Transposed matrix of U.
(Recalling from algebra, U*U^T = I)

The principal components are obtained from the columns of matrix U. The number of principal components to be selected depends on the specific problem and can be determined according to the percentage of variance explained by each component.

 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import skorch



In [7]:
#Generation of synthetic data with same dimensions as PAVI
# Train dimension before PCA:(196, 1010004)
# Train dimension after PCA:(196, 100)
# Input dimension to ridge:(196, 100), Output dimension of ridge:(196, 13)
#type = numpy.darray


np.random.seed(1)
x_aspavi_data = np.random.rand(196, 1000000).astype(np.float32)
y_aspavi_data = np.random.rand(196, 13).astype(np.float64)


X_tensor = torch.tensor(x_aspavi_data, dtype=torch.float32)
y_tensor = torch.tensor(y_aspavi_data, dtype=torch.float32)

# X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
input_dim = X_tensor.shape[1]
output_dim = y_tensor.shape[1]

hidden_dim = 100



In [8]:
#Model definition
class RidgeRegressionWDimRed(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, alpha=0.1, lamb = 0.1):
        super(RidgeRegressionWDimRed, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Linear(hidden_dim, output_dim)
        )
        self.alpha = alpha
        self.lamb = lamb
    
    def forward(self, x):
        return self.model(x)
    
    def orthogonal_regularization(self):
        rank_transform = self.model[0].weight @ self.model[0].weight.T
        identity = torch.eye(rank_transform.shape[0])
        return (self.lamb*(rank_transform - identity) ** 2).sum()


    def l2_regularization(self):
        l2_norm = torch.norm(self.model[1].weight)
        return self.alpha * l2_norm
    
    def total_loss(self):
        return self.orthogonal_regularization() + self.l2_regularization()

In [4]:
print(x_aspavi_data.shape[1], x_aspavi_data.dtype, type(x_aspavi_data))
print(y_aspavi_data.shape, y_aspavi_data.dtype, type(y_aspavi_data))





100000 float32 <class 'numpy.ndarray'>
(196, 13) float64 <class 'numpy.ndarray'>


In [7]:
#Training and optimizer steps
alpha = 0.1

model = RidgeRegressionWDimRed(input_dim, hidden_dim, output_dim, alpha, lamb=1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)



def train_model(model, optimizer, criterion, X_tensor, y_tensor, num_epochs=1000):
    for epoch in range(num_epochs):

        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor) + model.total_loss()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

train_model(model=model,optimizer=optimizer, criterion=criterion, X_tensor=X_tensor, y_tensor=y_tensor)

Epoch [1/1000], Loss: 45.0557
Epoch [2/1000], Loss: 44.5925
Epoch [3/1000], Loss: 44.3087
Epoch [4/1000], Loss: 44.0574
Epoch [5/1000], Loss: 43.8160
Epoch [6/1000], Loss: 43.5777
Epoch [7/1000], Loss: 43.3405
Epoch [8/1000], Loss: 43.1036
Epoch [9/1000], Loss: 42.8669
Epoch [10/1000], Loss: 42.6304
Epoch [11/1000], Loss: 42.3939
Epoch [12/1000], Loss: 42.1576
Epoch [13/1000], Loss: 41.9215
Epoch [14/1000], Loss: 41.6854
Epoch [15/1000], Loss: 41.4496
Epoch [16/1000], Loss: 41.2139
Epoch [17/1000], Loss: 40.9784
Epoch [18/1000], Loss: 40.7431
Epoch [19/1000], Loss: 40.5080
Epoch [20/1000], Loss: 40.2732
Epoch [21/1000], Loss: 40.0386
Epoch [22/1000], Loss: 39.8042
Epoch [23/1000], Loss: 39.5702
Epoch [24/1000], Loss: 39.3363
Epoch [25/1000], Loss: 39.1028
Epoch [26/1000], Loss: 38.8696
Epoch [27/1000], Loss: 38.6368
Epoch [28/1000], Loss: 38.4042
Epoch [29/1000], Loss: 38.1720
Epoch [30/1000], Loss: 37.9402
Epoch [31/1000], Loss: 37.7087
Epoch [32/1000], Loss: 37.4777
Epoch [33/1000], 

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

model.eval()
y_pred = model(X_test)

print(X_test.shape)
print(y_pred.shape)

# Calculate the mean squared error (MSE) on the test data
mse_loss = criterion(y_pred, y_test)
print(f"Mean Squared Error on Test Data: {mse_loss.item():.4f}")


torch.Size([40, 10000])
torch.Size([40, 13])
Mean Squared Error on Test Data: 0.0153


Implementing the model with skorch
(https://skorch.readthedocs.io/en/latest/user/customization.html)


In [21]:
#Model definition
class RidgeRegressionWDimRed(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, alpha=1):
        super(RidgeRegressionWDimRed, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Linear(hidden_dim, output_dim)
        )
        self.alpha = alpha

    
    def orthogonal_regularization(self):
        rank_transform = self.net[0].weight @ self.net[0].weight.T
        identity = torch.eye(rank_transform.shape[0])
        return ((rank_transform - identity) ** 2).sum()

    def l2_regularization(self):
        l2_norm = torch.norm(self.net[1].weight)
        return self.alpha * l2_norm
    
    def forward(self, x):
        return self.net(x)
    
    

In [9]:
class RidgeNet(skorch.NeuralNet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def get_loss(self, y_pred, y_true, X=None, training=False):
        loss = super().get_loss(y_pred, y_true, X=X, training=training)
        loss += self.module_.l2_regularization() + self.module_.orthogonal_regularization()
        return loss

In [12]:
print(input_dim, hidden_dim, output_dim)
model = RidgeNet(
        module= RidgeRegressionWDimRed,
        module__input_dim=input_dim,
        module__hidden_dim=hidden_dim,
        module__output_dim=output_dim,
        criterion=nn.MSELoss,
        optimizer=torch.optim.SGD,
        optimizer__lr=0.0001,
        max_epochs=5000,
        module__alpha=1,
        device='cuda' if torch.cuda.is_available() else 'cpu', 
        )
model.initialize()

1000000 100 13


<class '__main__.RidgeNet'>[initialized](
  module_=RidgeRegressionWDimRed(
    (model): Sequential(
      (0): Linear(in_features=1000000, out_features=100, bias=True)
      (1): Linear(in_features=100, out_features=13, bias=True)
    )
  ),
)

In [13]:
model.fit(X_tensor, y_tensor)

Re-initializing module because the following parameters were re-set: alpha, hidden_dim, input_dim, output_dim.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        6.8896        6.7508  3.3191
      2        6.7337        6.7738  3.4509
      3        6.7573        6.8296  3.4960
      4        6.8154        6.9680  3.3850
      5        6.9621        7.3015  3.5951
      6        7.3160        8.0639  3.7846
      7        8.1219        9.7155  3.7282
      8        9.8552       13.0660  3.0511
      9       13.3320       19.1759  2.5335
     10       19.5389       28.1840  2.4330
     11       28.3021       36.1598  2.5081
     12       35.2298       34.6111  2.7318
     13       32.3435       22.2971  3.4438
     14       19.9911       10.5718  2.5204
     15        9.4637        6.6462  2.3823
     16        6.8492        9.4820  2.8895
     17       10.6382       15.8020  2.5663